## Solvers' Testing

In [1]:
# Add locations of modules
push!(LOAD_PATH, "../src/")

3-element Array{Any,1}:
 "/Applications/Julia-0.6.app/Contents/Resources/julia/local/share/julia/site/v0.6"
 "/Applications/Julia-0.6.app/Contents/Resources/julia/share/julia/site/v0.6"      
 "../src/"                                                                         

## Coordinate Descent

## Test 1

In [2]:
using GraphicalModelLearning

srand(0)

# Sample considering a simple factor graph
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])
samples1 = sample(model, 5000)

# Sample using the Gibbs Sampler
n_samples = 100000
samples_time_series, nodes_updated_time_series = gibbs_sampling(model, n_samples)

# Every mth sample after burn in (Determining when burn-in happens?)
t_burn_in = 50000
m_samples = 10

samples_iid = samples_time_series[t_burn_in:m_samples:n_samples,:]

# Compare samples against those produced by direct method
using StatsBase
raw_binning = countmap(samples_iid)
samples2 = [ vcat(raw_binning[i], 2*digits(i, 2, model.varible_count)-1) for i in keys(raw_binning)]
samples2 = hcat(samples2...)'

# Other trends to check samples after a long time are decorrelated and belong to the stationary distribution

INFO: Recompiling stale cache file /Users/gogol/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.
INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate samples


8×4 Array{Int64,2}:
 1309  -1  -1  -1
  820  -1  -1   1
 1306   1   1   1
  182  -1   1  -1
  848   1   1  -1
  180   1  -1   1
  180  -1   1   1
  176   1  -1  -1

In [3]:
samples2[:,1]/sum(samples2[:,1])

8-element Array{Float64,1}:
 0.261748 
 0.163967 
 0.261148 
 0.0363927
 0.169566 
 0.0359928
 0.0359928
 0.035193 

In [4]:
samples1[:,1]/sum(samples1[:,1])

8-element Array{Float64,1}:
 0.2614
 0.1668
 0.2532
 0.0362
 0.1786
 0.0354
 0.0348
 0.0336

### Learning on iid samples

In [5]:
# Can the FactorGraph be recovered?
@time learned1 = learn(samples1)
@time learned2 = learn(samples2)

@time learned_cgd_rise1 = learn_cgd(samples1)
@time learned_cgd_rise2 = learn_cgd(samples2)

@time learned_cgd_rple1 = learn_cgd(samples1, RPLE())
@time learned_cgd_rple2 = learn_cgd(samples2, RPLE())

@time learned_cgd_logrise1 = learn_cgd(samples1, logRISE())
@time learned_cgd_logrise2 = learn_cgd(samples2, logRISE())

println(learned1)
println(learned2)

println(learned_cgd_rise1)
println(learned_cgd_rise2)

println(learned_cgd_rple1)
println(learned_cgd_rple2)

println(learned_cgd_logrise1)
println(learned_cgd_logrise2)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  7.718836 seconds (5.18 M allocations: 260.354 MiB, 1.27% gc time)
  0.019887 seconds (5.27 k allocations: 363.656 KiB)


INFO: using coordinate gradient solver for RISE


  2.475346 seconds (1.00 M allocations: 52.261 MiB, 0.79% gc time)
  0.001544 seconds (6.55 k allocations: 159.938 KiB)


INFO: using coordinate gradient solver for RISE
INFO: using coordinate gradient solver for RPLE


  0.321572 seconds (188.65 k allocations: 8.759 MiB, 2.36% gc time)
  0.006880 seconds (30.98 k allocations: 809.281 KiB)


INFO: using coordinate gradient solver for RPLE
INFO: using coordinate gradient solver for logRISE


  0.340720 seconds (186.64 k allocations: 9.437 MiB)
  0.001670 seconds (9.52 k allocations: 208.656 KiB)


INFO: using coordinate gradient solver for logRISE


[-0.00385463 0.880962 0.0986432; 0.880962 0.00991948 0.0762026; 0.0986432 0.0762026 -0.0209155]
[-0.000826041 0.863831 0.104378; 0.863831 0.00735092 0.0961333; 0.104378 0.0961333 -0.0071306]
[0.0 0.553234 0.115521; 0.553234 0.0 0.101588; 0.115521 0.101588 -0.00780131]
[0.0 0.82061 0.0830133; 0.82061 0.0 0.120779; 0.0830133 0.120779 0.0]
[0.0 0.674117 0.119962; 0.674117 0.0 0.0801117; 0.119962 0.0801117 -0.0142314]
[0.0 0.691204 0.129965; 0.691204 0.0 0.100219; 0.129965 0.100219 -0.000428487]
[0.0 0.669624 0.108235; 0.669624 0.0 0.0791189; 0.108235 0.0791189 0.0]
[0.0 0.805056 0.0994766; 0.805056 0.0 0.105919; 0.0994766 0.105919 0.0]


In [5]:
@time learned_cd_rise1 = learn(samples1, RISE(), CD(1e-12))
@time learned_cd_rise2 = learn(samples2, RISE(), CD(1e-12))

@time learned_cd_rise1 = learn(samples1, logRISE(), CD(1e-12))
@time learned_cd_rise2 = learn(samples2, logRISE(), CD(1e-12))

#println(learned_cd_rise1)
#println(learned_cd_rise2)

INFO: using coordinate descent solver for RISE


  0.671387 seconds (424.58 k allocations: 20.727 MiB, 1.30% gc time)
  0.000718 seconds (16.33 k allocations: 289.219 KiB)


INFO: using coordinate descent solver for RISE


  0.391253 seconds (237.72 k allocations: 11.212 MiB)
  0.001147 seconds (19.34 k allocations: 341.500 KiB)


INFO: using coordinate descent solver for logRISE
INFO: using coordinate descent solver for logRISE


3×3 Array{Float64,2}:
 -0.000790879  0.857271     0.0966116
  0.857271     0.00732506   0.0897174
  0.0966116    0.0897174   -0.0070327

### Learning on correlated samples -- T-regime

In [2]:
using GraphicalModelLearning

srand(0)

# Define Graphical Model
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.0 0.9; 0.9 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])

n_samples = 5000
# Plain sampling
samples1 = sample(model, n_samples)

# Gibbs Samples
samples_T, samples_mixed = gibbs_sampling(model, n_samples, T_regime())
#gibbs_output = gibbs_sampling(model, n_samples, T_regime())
#samples_T = gibbs_output[1]
#samples_mixed = gibbs_output[2]
print("Done sampling")

INFO: Recompiling stale cache file /Users/gogol/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.
INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate T-regime samples


Done sampling

In [3]:
samples_T

48×8 Array{Int64,2}:
  54  1   1  -1   1  -1  -1   1
  45  2   1   1  -1   1  -1  -1
  37  3   1  -1   1   1  -1   1
  45  2  -1  -1   1  -1   1   1
  55  1   1   1   1  -1   1   1
  10  1   1  -1   1   1  -1   1
 182  3  -1  -1   1  -1  -1  -1
  48  1  -1  -1   1   1  -1   1
 352  1  -1  -1  -1  -1  -1  -1
 182  3   1   1  -1   1   1   1
  35  3  -1   1   1  -1   1   1
  47  1   1   1  -1  -1   1  -1
 339  1   1   1   1   1   1   1
   ⋮                  ⋮        
 126  3   1   1  -1   1   1  -1
 114  3  -1  -1   1  -1  -1   1
  39  1  -1   1  -1   1   1  -1
   5  1  -1   1   1  -1   1   1
  27  3  -1   1  -1  -1   1   1
  64  1  -1  -1  -1   1  -1  -1
   7  1  -1   1  -1  -1   1  -1
  55  1  -1   1   1   1   1   1
  47  2  -1   1   1  -1  -1   1
  60  2   1  -1  -1   1   1  -1
  29  3   1  -1  -1   1  -1   1
  10  1   1  -1  -1   1  -1  -1

In [4]:
samples_mixed

8×4 Array{Int64,2}:
 1323  -1  -1  -1
  842  -1  -1   1
 1214   1   1   1
  164  -1   1  -1
  888   1   1  -1
  189   1  -1   1
  174  -1   1   1
  206   1  -1  -1

In [5]:
@time learned3 = learn_glauber_dynamics(samples_T, RISE())

INFO: using JuMP for RISE



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  7.855883 seconds (5.29 M allocations: 267.146 MiB, 1.34% gc time)


3×3 Array{Float64,2}:
 0.0300259   0.857774    0.051578 
 0.857774   -0.0392253   0.0903428
 0.051578    0.0903428  -0.0326325

In [6]:
@time learned3 = learn_glauber_dynamics(samples_T, logRISE())

  0.262247 seconds (98.31 k allocations: 5.002 MiB)


INFO: using JuMP for logRISE


3×3 Array{Float64,2}:
 0.0295788   0.851016    0.0443866
 0.851016   -0.0391159   0.0833681
 0.0443866   0.0833681  -0.0328581

In [7]:
@time learned4 = learn_glauber_dynamics(samples_T, RISE(), CD(1e-12))

INFO: using coordinate descent solver for RISE to learn Glauber dyamics


  0.602921 seconds (413.70 k allocations: 19.933 MiB, 1.40% gc time)


3×3 Array{Float64,2}:
 0.0300259   0.857774    0.051655 
 0.857774   -0.0392252   0.0902334
 0.051655    0.0902334  -0.032638 

In [10]:
@time learned4 = learn_glauber_dynamics(samples_T, logRISE(), CD(1e-12))

INFO: using coordinate descent solver for logRISE to learn Glauber dyamics


  0.532190 seconds (364.20 k allocations: 16.879 MiB, 1.68% gc time)


3×3 Array{Float64,2}:
 0.0295788   0.851016    0.0443828
 0.851016   -0.0391159   0.0833734
 0.0443828   0.0833734  -0.0328578

In [9]:
learned4 = learn(samples_mixed)

3×3 Array{Float64,2}:
 0.0319933   0.856709    0.0661006
 0.856709   -0.0437774   0.100655 
 0.0661006   0.100655   -0.0307446

In [61]:
@time learned4 = learn_glauber_dynamics_cgd(samples_T)

  0.011214 seconds (91.09 k allocations: 1.962 MiB)


INFO: using coordinate gradient solver for RISE


3×3 Array{Float64,2}:
 0.0137192   0.856471    0.0612671
 0.856471   -0.027355    0.0979819
 0.0612671   0.0979819  -0.0173443

In [11]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, RISE(), 8.0, 1e-10)

INFO: using coordinate gradient solver for RISE


  0.018075 seconds (113.89 k allocations: 2.454 MiB)


3×3 Array{Float64,2}:
 0.0137234   0.856503    0.0613336
 0.856503   -0.0273728   0.0978732
 0.0613336   0.0978732  -0.0173494

In [12]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, RISE(), 8.0, 1e-11)

  0.017757 seconds (129.83 k allocations: 2.797 MiB)


INFO: using coordinate gradient solver for RISE


3×3 Array{Float64,2}:
 0.0137237   0.85651     0.0613362
 0.85651    -0.0273761   0.0978637
 0.0613362   0.0978637  -0.0173498

In [13]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, RPLE())

INFO: using coordinate gradient solver for RPLE


  0.317433 seconds (275.36 k allocations: 11.259 MiB, 3.64% gc time)


3×3 Array{Float64,2}:
 0.0191458   0.860655   0.0656544
 0.860655   -0.030531   0.101443 
 0.0656544   0.101443  -0.0236575

In [14]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, RPLE(), 8.0, 1e-10)

  0.022029 seconds (170.83 k allocations: 3.840 MiB)


INFO: using coordinate gradient solver for RPLE


3×3 Array{Float64,2}:
 0.0192199   0.860844    0.0656448
 0.860844   -0.0305775   0.101365 
 0.0656448   0.101365   -0.0236612

In [15]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, logRISE())

INFO: using coordinate gradient solver for logRISE


  0.431719 seconds (381.82 k allocations: 15.108 MiB, 1.86% gc time)


3×3 Array{Float64,2}:
 0.00453186   0.848603    0.0532006
 0.848603    -0.0189449   0.0907966
 0.0532006    0.0907966  -0.0046365

In [16]:
@time learned5 = learn_glauber_dynamics_cgd(samples_T, logRISE(), 8.0, 1e-10)

  0.024551 seconds (245.76 k allocations: 4.840 MiB)


INFO: using coordinate gradient solver for logRISE


3×3 Array{Float64,2}:
 0.00453198   0.848603    0.0533993 
 0.848603    -0.0189449   0.0906347 
 0.0533993    0.0906347  -0.00464917

### Learning on correlated samples -- M-regime

In [2]:
using GraphicalModelLearning

srand(0)

# Define Graphical Model
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])
#model = FactorGraph([0.0 0.3 0.2; 0.3 0.0 0.4; 0.2 0.4 0.0])

n_samples = 10000
# Plain sampling
samples1 = sample(model, n_samples)

# Gibbs Samples
samples_M, samples_mixed = gibbs_sampling(model, n_samples, M_regime())
#gibbs_output = gibbs_sampling(model, n_samples, T_regime())
#samples_T = gibbs_output[1]
#samples_mixed = gibbs_output[2]
print("Done sampling")

INFO: Recompiling stale cache file /Users/gogol/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.
INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate M-regime samples


Done sampling

In [3]:
samples_M

48×8 Array{Int64,2}:
 211  3  -1   1   1  -1   1  -1
 268  3  -1  -1  -1  -1  -1  -1
  49  2   1  -1   1   1  -1   1
  53  2  -1   1  -1  -1   1  -1
 198  3   1  -1  -1   1  -1  -1
  57  2   1   1   1   1  -1   1
 151  3   1   1   1   1   1  -1
  65  1   1   1  -1  -1   1  -1
 380  1   1  -1   1  -1  -1   1
 262  3  -1  -1   1  -1  -1  -1
 336  1  -1   1  -1   1   1  -1
  49  1   1   1   1  -1   1   1
 368  2  -1  -1   1  -1  -1   1
   ⋮                  ⋮        
 172  3  -1  -1  -1  -1  -1   1
  55  1  -1   1   1  -1   1   1
 344  1  -1   1   1   1   1   1
  52  1  -1  -1  -1   1  -1  -1
 226  3   1   1   1   1   1   1
  72  2   1  -1  -1   1  -1  -1
  64  2  -1   1   1  -1   1   1
  56  1   1  -1  -1   1  -1  -1
 197  3  -1   1  -1  -1   1  -1
 224  3  -1   1   1  -1   1   1
 347  1  -1  -1   1  -1  -1   1
 222  3   1  -1   1   1  -1  -1

In [13]:
samples_T

48×8 Array{Int64,2}:
  25  3  -1   1   1  -1   1  -1
 301  3  -1  -1  -1  -1  -1  -1
  10  2   1  -1   1   1  -1   1
   8  2  -1   1  -1  -1   1  -1
  43  3   1  -1  -1   1  -1  -1
  55  2   1   1   1   1  -1   1
 175  3   1   1   1   1   1  -1
  47  1   1   1  -1  -1   1  -1
  54  1   1  -1   1  -1  -1   1
 182  3  -1  -1   1  -1  -1  -1
  39  1  -1   1  -1   1   1  -1
  55  1   1   1   1  -1   1   1
 219  2  -1  -1   1  -1  -1   1
   ⋮                  ⋮        
 175  3  -1  -1  -1  -1  -1   1
   5  1  -1   1   1  -1   1   1
  55  1  -1   1   1   1   1   1
  64  1  -1  -1  -1   1  -1  -1
 250  3   1   1   1   1   1   1
  14  2   1  -1  -1   1  -1  -1
   7  2  -1   1   1  -1   1   1
  10  1   1  -1  -1   1  -1  -1
  31  3  -1   1  -1  -1   1  -1
  35  3  -1   1   1  -1   1   1
 233  1  -1  -1   1  -1  -1   1
  30  3   1  -1   1   1  -1  -1

In [4]:
samples_mixed

8×4 Array{Int64,2}:
 2030  -1  -1  -1
 1808  -1  -1   1
 1889   1   1   1
  652  -1   1  -1
 1621   1   1  -1
  663   1  -1   1
  672  -1   1   1
  665   1  -1  -1

In [5]:
@time learned1 = learn_glauber_dynamics(samples_M)

INFO: using JuMP for RISE



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  7.603142 seconds (5.26 M allocations: 264.256 MiB, 1.33% gc time)


3×3 Array{Float64,2}:
 -0.0136387   0.869581   0.0746245 
  0.869581   -0.0220767  0.0874464 
  0.0746245   0.0874464  0.00346713

In [6]:
@time learned1 = learn_glauber_dynamics(samples_M, logRISE())

  0.260733 seconds (98.14 k allocations: 4.995 MiB)


INFO: using JuMP for logRISE


3×3 Array{Float64,2}:
 -0.0137665   0.863962   0.0671916 
  0.863962   -0.0221965  0.0801299 
  0.0671916   0.0801299  0.00310045

In [13]:
@time learned1 = learn_glauber_dynamics(samples_M, RISE(), CD(1e-12))

  0.001472 seconds (24.70 k allocations: 429.656 KiB)


INFO: using coordinate descent solver for RISE to learn Glauber dyamics


3×3 Array{Float64,2}:
 -0.0136387   0.869581   0.0746245 
  0.869581   -0.0220767  0.0874464 
  0.0746245   0.0874464  0.00346711

In [14]:
@time learned1 = learn_glauber_dynamics(samples_M, logRISE(), CD(1e-12))

  0.001329 seconds (21.01 k allocations: 367.844 KiB)


INFO: using coordinate descent solver for logRISE to learn Glauber dyamics


3×3 Array{Float64,2}:
 -0.0137665   0.863962   0.0671915 
  0.863962   -0.0221965  0.0801298 
  0.0671915   0.0801298  0.00310044

In [13]:
learned2 = learn(samples_mixed)

3×3 Array{Float64,2}:
 -0.0219749   0.497947   0.0213356 
  0.497947   -0.0235339  0.0296025 
  0.0213356   0.0296025  0.00821935

In [67]:
@time learned1 = learn_glauber_dynamics_cgd(samples_M)

  0.002986 seconds (18.16 k allocations: 399.094 KiB)


INFO: using coordinate gradient solver for RISE


3×3 Array{Float64,2}:
 0.0         0.182647   0.00922189
 0.182647    0.0        0.0208984 
 0.00922189  0.0208984  0.0       

In [66]:
@time learned2 = learn(samples_mixed)

  0.028327 seconds (6.20 k allocations: 397.609 KiB)


3×3 Array{Float64,2}:
 0.0132895   0.496226    0.0197846
 0.496226   -0.0148107   0.0374478
 0.0197846   0.0374478  -0.012612 

### Large N Graphical Models

In [50]:
using GraphicalModelLearning

# Create a random graphical model
N_spins = 10
N_samples = 100000
d_max = 3
density_gm = d_max/N_spins

F1 = sprand(N_spins,N_spins,density_gm)
F1 = 0.5*(F1 + transpose(F1))

F1 = full(F1)

model = FactorGraph(F1)
@time samples1 = sample(model, N_samples)

INFO: assuming spin alphabet


  0.044406 seconds (88.44 k allocations: 6.481 MiB)


1009×11 Array{Int64,2}:
  12  -1   1  -1  -1   1   1  -1  -1   1  -1
  12   1  -1   1   1   1  -1  -1  -1  -1  -1
 166  -1   1  -1   1  -1  -1   1  -1  -1  -1
   5   1  -1  -1   1  -1  -1  -1   1   1   1
  26  -1  -1  -1  -1   1   1  -1   1  -1  -1
   7  -1  -1   1   1   1   1   1  -1   1   1
  11   1  -1   1   1   1  -1  -1  -1   1  -1
   2  -1   1   1   1   1   1  -1  -1   1  -1
   2   1  -1  -1   1  -1   1   1  -1   1   1
 258   1   1   1   1  -1  -1   1   1   1   1
  47  -1   1  -1  -1  -1   1   1  -1   1  -1
  70  -1   1  -1  -1  -1   1   1  -1  -1   1
  25   1   1  -1  -1   1   1  -1  -1  -1   1
   ⋮                   ⋮                   ⋮
   4  -1  -1   1   1  -1   1  -1  -1   1   1
  10  -1  -1  -1   1   1   1   1  -1  -1   1
  11  -1  -1   1  -1   1   1   1   1   1   1
  16  -1   1  -1   1  -1   1   1  -1  -1   1
   3   1  -1   1  -1   1  -1   1  -1   1  -1
  83  -1   1  -1   1   1   1  -1   1  -1  -1
   3   1  -1  -1  -1  -1  -1   1  -1   1  -1
  24  -1  -1   1  -1  -1   1  -

In [51]:
@time learned1 = learn(samples1)
@time learned2 = learn(samples1)

@time learned_cgd_rise1 = learn_cgd(samples1)
@time learned_cgd_rise2 = learn_cgd(samples1)

  1.022895 seconds (1.66 M allocations: 183.636 MiB, 17.60% gc time)
  0.876679 seconds (1.66 M allocations: 183.637 MiB, 2.82% gc time)


INFO: using coordinate gradient solver for RISE


  1.288674 seconds (48.20 M allocations: 806.876 MiB, 6.13% gc time)


INFO: using coordinate gradient solver for RISE


  1.401945 seconds (55.69 M allocations: 932.224 MiB, 5.79% gc time)


10×10 Array{Float64,2}:
 0.00199484  0.0         0.427628   …  0.00865155   0.246083    0.00544317
 0.0         0.614433    0.0052952     0.0415487    0.105921    0.00591136
 0.427628    0.0052952   0.13687       0.0130668    0.165301    0.0       
 0.228291    0.0         0.0           0.453157     0.0607459   0.0576528 
 0.215814    0.0468684   0.0           0.166347     0.0338914   0.38767   
 0.21193     0.0968438   0.126132   …  0.000451817  0.0         0.0       
 0.0702882   0.0         0.0           0.0247585    0.0        -0.0143422 
 0.00865155  0.0415487   0.0130668     0.16428      0.235174    0.0612548 
 0.246083    0.105921    0.165301      0.235174     0.0761064   0.536904  
 0.00544317  0.00591136  0.0           0.0612548    0.536904    0.0       

In [52]:
F1

10×10 Array{Float64,2}:
 0.0        0.0        0.426423  …  0.0        0.250543   0.0      
 0.0        0.617477   0.0          0.0        0.113561   0.0      
 0.426423   0.0        0.149453     0.0        0.166875   0.0      
 0.240953   0.0        0.0          0.439501   0.055724   0.0333312
 0.21271    0.0597777  0.0          0.215525   0.0357375  0.417942 
 0.216005   0.10439    0.126462  …  0.0        0.0        0.0      
 0.0691028  0.0        0.0          0.0297661  0.0        0.0      
 0.0        0.0        0.0          0.242806   0.277266   0.0      
 0.250543   0.113561   0.166875     0.277266   0.0777402  0.554626 
 0.0        0.0        0.0          0.0        0.554626   0.0      

In [43]:
using GraphicalModelLearning

# Create a random graphical model
N_spins = 10
N_samples = 100000
d_max = 3
density_gm = d_max/N_spins

F1 = sprand(N_spins,N_spins,density_gm)
F1 = 0.5*(F1 + transpose(F1))

F1 = sign.(full(F1))

model = FactorGraph(F1)
@time samples1 = sample(model, N_samples)

  0.032342 seconds (31.83 k allocations: 3.786 MiB)


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] sign(::Array{Float64,2}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/gogol/.julia/v0.6/IJulia/src/execute_request.jl:158
 [5] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/gogol/.julia/v0.6/Compat/src/Compat.jl:386
 [6] eventloop(::ZMQ.Socket) at /Users/gogol/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[43], in expression starting on line 12
INFO: assuming spin alphabet


17×11 Array{Int64,2}:
     2  -1  -1  -1  -1  -1   1  -1  -1  -1  -1
     4  -1   1  -1  -1  -1  -1  -1  -1  -1  -1
   239  -1   1   1   1   1   1   1   1   1   1
     2  -1  -1  -1  -1   1  -1  -1  -1  -1  -1
    38   1   1   1   1   1  -1   1   1   1   1
     3   1   1   1   1  -1   1   1   1   1  -1
  1849  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
     8   1   1   1   1   1   1  -1   1   1   1
    27   1   1  -1   1   1   1   1   1   1   1
     1   1   1  -1   1  -1   1   1   1   1   1
     4  -1   1  -1   1   1   1   1   1   1   1
    25   1   1   1   1  -1   1   1   1   1   1
 97759   1   1   1   1   1   1   1   1   1   1
     3  -1  -1  -1  -1  -1  -1  -1  -1  -1   1
    33   1   1   1   1   1   1   1   1   1  -1
     1   1  -1   1   1   1   1   1   1   1   1
     2   1  -1  -1  -1  -1  -1  -1  -1  -1  -1

In [44]:
@time learned1 = learn(samples1)
@time learned2 = learn(samples1)

@time learned_cgd_rise1 = learn_cgd(samples1)
@time learned_cgd_rise2 = learn_cgd(samples1)

#@time learned_sgd_rise1 = learn_sgd(samples1)
#@time learned_sgd_rise2 = learn_sgd(samples1)

  0.137082 seconds (46.04 k allocations: 4.708 MiB)
  0.123117 seconds (46.04 k allocations: 4.708 MiB)


INFO: using coordinate gradient solver for RISE


  5.567424 seconds (71.29 M allocations: 1.281 GiB, 3.26% gc time)


INFO: using coordinate gradient solver for RISE


  6.686495 seconds (86.60 M allocations: 1.555 GiB, 3.20% gc time)


10×10 Array{Float64,2}:
 0.104432  0.478404  0.36409   0.296252  …  0.739646  0.340798  0.372659
 0.478404  1.02587   0.435044  0.433103     0.49594   0.426568  0.345894
 0.36409   0.435044  0.487701  0.40621      0.414062  0.500734  0.30407 
 0.296252  0.433103  0.40621   0.479149     0.818452  0.578383  0.47726 
 0.225612  0.280649  0.680348  0.441969     0.373381  0.729973  0.883473
 0.250224  0.962323  0.402798  0.369127  …  0.711565  0.406131  0.25307 
 0.262815  0.72944   0.474294  0.434483     0.511948  0.531137  0.33579 
 0.739646  0.49594   0.414062  0.818452     0.151754  0.492866  0.606765
 0.340798  0.426568  0.500734  0.578383     0.492866  0.136219  0.380493
 0.372659  0.345894  0.30407   0.47726      0.606765  0.380493  0.548194

In [45]:
F1

10×10 Array{Float64,2}:
 0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0
 1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  0.0
 1.0  1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0
 0.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
 0.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0

In [48]:
learned1

10×10 Array{Float64,2}:
 0.363398     0.000152139  0.998232     …  0.269543  0.269543  3.38883e-5 
 0.000152139  1.64555      0.000333802     0.637412  0.637639  9.76061e-5 
 0.998232     0.000333802  0.829737        0.186376  0.186355  7.22393e-5 
 0.269543     0.637412     0.186376        2.17459   2.17097   0.290854   
 3.57279e-5   0.000119345  1.14516         0.111397  0.111395  1.3384     
 3.36551e-5   0.000110999  0.000107648  …  0.512129  0.512127  4.5451e-5  
 0.000179216  0.000862972  0.000196304     0.582907  0.582752  0.000247517
 0.269543     0.637412     0.186376        1.98236   2.17313   0.290854   
 0.269543     0.637639     0.186355        2.17313   1.98236   0.290853   
 3.38883e-5   9.76061e-5   7.22393e-5      0.290854  0.290853  0.858315   

In [47]:
learned_cgd_rise2

10×10 Array{Float64,2}:
 0.104432  0.478404  0.36409   0.296252  …  0.739646  0.340798  0.372659
 0.478404  1.02587   0.435044  0.433103     0.49594   0.426568  0.345894
 0.36409   0.435044  0.487701  0.40621      0.414062  0.500734  0.30407 
 0.296252  0.433103  0.40621   0.479149     0.818452  0.578383  0.47726 
 0.225612  0.280649  0.680348  0.441969     0.373381  0.729973  0.883473
 0.250224  0.962323  0.402798  0.369127  …  0.711565  0.406131  0.25307 
 0.262815  0.72944   0.474294  0.434483     0.511948  0.531137  0.33579 
 0.739646  0.49594   0.414062  0.818452     0.151754  0.492866  0.606765
 0.340798  0.426568  0.500734  0.578383     0.492866  0.136219  0.380493
 0.372659  0.345894  0.30407   0.47726      0.606765  0.380493  0.548194